In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50V2

tf.__version__

'2.5.0'

In [ ]:
!git clone https://github.com/CartagenaMinas/Melanomas.git

Cloning into 'Melanomas'...
remote: Enumerating objects: 43603, done.
remote: Total 43603 (delta 0), reused 0 (delta 0), pack-reused 43603
Receiving objects: 100% (43603/43603), 219.76 MiB | 36.24 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (44116/44116), done.


In [ ]:
import os
os.listdir("Melanomas")

['data', 'Keras_baseline.ipynb', 'exploracion.ipynb', '.git']

In [ ]:
PATH="Melanomas/data"
os.listdir(PATH)

['subset.csv',
 'jpeg224',
 'train_split.csv',
 'sample_submission.csv',
 'val_split.csv',
 'train.csv',
 'test.csv']

In [ ]:
import pandas as pd
train=pd.read_csv(f"{PATH}/subset.csv")
val=pd.read_csv(f"{PATH}/val_split.csv")
train.shape,val.shape

((2220, 8), (10932, 8))

In [ ]:
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_0533349,IP_5208504,female,45.0,lower extremity,unknown,benign,0
1,ISIC_8814612,IP_0414408,male,50.0,torso,unknown,benign,0
2,ISIC_6515241,IP_6245507,male,45.0,lower extremity,unknown,benign,0
3,ISIC_5075261,IP_2117218,male,40.0,upper extremity,unknown,benign,0
4,ISIC_2624460,IP_1969685,male,50.0,torso,unknown,benign,0


In [ ]:
BATCH_SIZE=64

def decode(name,label):
  img=tf.io.read_file(name)#lo lee en byts
  img=tf.image.decode_jpeg(img,channels=3)#lo carga como imagen
  img=tf.cast(img,tf.float32)#/255#cambia la presicion
  return img,label

def load_ds(df):
  imgs, labels=df["image_name"].values,df["target"].values#Creo una lista 
  imgs=[f"{PATH}/jpeg224/train/{name}.jpg" for name in imgs]#Creo una lista con mis rutas
  ds=tf.data.Dataset.from_tensor_slices((imgs,labels))
  ds=ds.map(decode)
  ds=ds.shuffle(2048)
  ds=ds.batch(BATCH_SIZE)
  return ds

In [ ]:
train_ds=load_ds(train)
val_ds=load_ds(val)

In [ ]:
#for img, label in train_ds.take(1):
#  print(img, label)

In [ ]:
IMAGE_SIZE= (224,224,3)

encoder=ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights="imagenet"
)
encoder.trainable=False #No aplicar los gradientes calculados

inputs=keras.Input(shape=IMAGE_SIZE)
x=keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x=encoder(x,training=False)#No calcula los gradientes
x=keras.layers.GlobalAveragePooling2D()(x)
outputs=keras.layers.Dense(1,activation="sigmoid")(x)
model=keras.Model(inputs,outputs)
model.summary()

94674944/94668760 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.AUC(name="auc")]
)

In [ ]:
model.fit(train_ds,epochs=10,validation_data=val_ds,validation_steps=10)

Epoch 1/10
35/35 [==============================] - 48s 339ms/step - loss: 0.1625 - auc: 0.4701 - val_loss: 0.1398 - val_auc: 0.4684
Epoch 2/10
35/35 [==============================] - 11s 275ms/step - loss: 0.1005 - auc: 0.5350 - val_loss: 0.1018 - val_auc: 0.5521
Epoch 3/10
35/35 [==============================] - 11s 281ms/step - loss: 0.0858 - auc: 0.6899 - val_loss: 0.0940 - val_auc: 0.7549
Epoch 4/10
35/35 [==============================] - 11s 280ms/step - loss: 0.0785 - auc: 0.7660 - val_loss: 0.0979 - val_auc: 0.6673
Epoch 5/10
35/35 [==============================] - 11s 283ms/step - loss: 0.0723 - auc: 0.8384 - val_loss: 0.0938 - val_auc: 0.7223
Epoch 6/10
35/35 [==============================] - 11s 292ms/step - loss: 0.0686 - auc: 0.8559 - val_loss: 0.0977 - val_auc: 0.7663
Epoch 7/10
35/35 [==============================] - 11s 291ms/step - loss: 0.0648 - auc: 0.8845 - val_loss: 0.0770 - val_auc: 0.8620
Epoch 8/10
35/35 [==============================] - 11s 289ms/step - 

In [ ]:
test=pd.read_csv(f"{PATH}/test.csv")
test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [ ]:
BATCH_SIZE=256

def decode_test(name):
  img=tf.io.read_file(name)#lo lee en byts
  img=tf.image.decode_jpeg(img,channels=3)#lo carga como imagen
  img=tf.cast(img,tf.float32)#/255#cambia la presicion
  return img

def load_test_ds(df):
  imgs=df["image_name"].values#Creo una lista 
  imgs=[f"{PATH}/jpeg224/test/{name}.jpg" for name in imgs]#Creo una lista con mis rutas
  ds=tf.data.Dataset.from_tensor_slices(imgs)
  ds=ds.map(decode_test)
  ds=ds.batch(BATCH_SIZE)
  return ds

In [ ]:
test_ds=load_test_ds(test)

In [ ]:
preds=model.predict(test_ds)

In [ ]:
preds

array([[0.00456984],
       [0.00227686],
       [0.00875383],
       ...,
       [0.02165064],
       [0.0040192 ],
       [0.02436254]], dtype=float32)

In [ ]:
submission=pd.DataFrame({
    "image_name":test["image_name"].values,"target":preds.ravel()
})
submission.head()

,image_name,target
0,ISIC_0052060,0.004570
1,ISIC_0052349,0.002277
2,ISIC_0058510,0.008754
3,ISIC_0073313,0.000614
4,ISIC_0073502,0.009814


In [ ]:
submission.to_csv("sumission1.csv",index=False)

In [ ]:
model.save("model_1.h5")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
